In [ ]:
import keras #DeepLearning Library
import numpy as np 
import pandas as pd

ModuleNotFoundError: No module named 'keras'

In [ ]:
from hyperopt import STATUS_OK,tpe,hp,Trials,fmin  ## This library is used mainly in Hyper parameter tuning in Depp Learning projects

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [7]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")

In [8]:
##Creating the ANN (Artificial Neural Network)

#Split the data
train ,test = train_test_split(data,test_size=0.20)

In [ ]:
# Create the Training and Testing data for both X and Y values

train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()
test_x = test.drop(['quality'],axis=1).values
test_y = test['quality'].values.ravel()
